<a href="https://colab.research.google.com/github/santhimaddipudi/Python-work/blob/master/Python_polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
df=pd.read_csv("/content/sample_data/Iris.csv")
df_filtered = df[df["SepalLengthCm"] > 5].groupby("Species").agg('sum')  # Filter rows where 'SepalLengthCm' is greater than 5
df_filtered.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,,
Iris-setosa,564,116.9,81.7,33.2,6.1
Iris-versicolor,3562,281.9,131.8,202.9,63.3
Iris-virginica,6168,324.5,146.2,273.1,99.6


In [5]:
import polars as pl
pl.__version__

'1.21.0'

In [6]:
#operations that are supported on materialized dataframes but not on lazy dataframes.
df_ops = set(x for x in dir(pl.DataFrame()) if not x.startswith('_'))
lazy_ops = set(x for x in dir(pl.LazyFrame()) if not x.startswith('_'))
print(sorted(df_ops - lazy_ops))

['corr', 'drop_in_place', 'equals', 'estimated_size', 'extend', 'flags', 'fold', 'get_column', 'get_column_index', 'get_columns', 'glimpse', 'hash_rows', 'height', 'hstack', 'insert_column', 'is_duplicated', 'is_empty', 'is_unique', 'item', 'iter_columns', 'iter_rows', 'iter_slices', 'map_rows', 'max_horizontal', 'mean_horizontal', 'min_horizontal', 'n_chunks', 'n_unique', 'partition_by', 'pivot', 'plot', 'product', 'rechunk', 'replace_column', 'row', 'rows', 'rows_by_key', 'sample', 'shape', 'shrink_to_fit', 'style', 'sum_horizontal', 'to_arrow', 'to_dict', 'to_dicts', 'to_dummies', 'to_init_repr', 'to_jax', 'to_numpy', 'to_pandas', 'to_series', 'to_struct', 'to_torch', 'transpose', 'unstack', 'upsample', 'vstack', 'write_avro', 'write_clipboard', 'write_csv', 'write_database', 'write_delta', 'write_excel', 'write_ipc', 'write_ipc_stream', 'write_json', 'write_ndjson', 'write_parquet']


In [7]:
import polars as pl
#lazy dataframe
q = (
    pl.scan_csv("/content/sample_data/Iris.csv")
    .filter(pl.col("SepalLengthCm") > 5)
    .group_by("Species")
    .agg(pl.all().sum())
)

df = q.collect()

df

Species,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
str,i64,f64,f64,f64,f64
"""Iris-virginica""",6168,324.5,146.2,273.1,99.6
"""Iris-versicolor""",3562,281.9,131.8,202.9,63.3
"""Iris-setosa""",564,116.9,81.7,33.2,6.1


In [8]:
#for lazy dataframe it wont show the data
df1=pl.scan_csv("/content/sample_data/Iris.csv").filter(pl.col("SepalLengthCm") > 5)
print(df1)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

FILTER [(col("SepalLengthCm")) > (5.0)] FROM
  Csv SCAN [/content/sample_data/Iris.csv]
  PROJECT */6 COLUMNS


In [9]:
col=pl.col("SepalLengthCm")
col_ops = set(x for x in dir(col) if not x.startswith('_'))
print(col_ops)

{'any', 'bitwise_and', 'mean', 'cbrt', 'reinterpret', 'tail', 'is_not_null', 'is_infinite', 'cot', 'struct', 'rle_id', 'shuffle', 'rechunk', 'round', 'rolling_mean_by', 'agg_groups', 'bitwise_or', 'arcsinh', 'filter', 'truediv', 'is_finite', 'is_nan', 'hist', 'eq', 'inspect', 'sub', 'dot', 'rolling_min_by', 'add', 'tan', 'repeat_by', 'log10', 'ne_missing', 'rolling_max', 'register_plugin', 'sinh', 'count', 'rolling_median', 'diff', 'where', 'over', 'head', 'product', 'ewm_mean', 'dt', 'cosh', 'ge', 'forward_fill', 'gather_every', 'and_', 'arccosh', 'rolling_sum_by', 'std', 'gt', 'ewm_mean_by', 'quantile', 'abs', 'append', 'rle', 'first', 'degrees', 'bottom_k_by', 'arg_sort', 'arg_min', 'rolling_mean', 'cos', 'floor', 'hash', 'sample', 'shrink_dtype', 'reverse', 'entropy', 'arctanh', 'round_sig_figs', 'clip', 'limit', 'rolling_var_by', 'xor', 'mod', 'peak_max', 'cut', 'deserialize', 'cum_prod', 'is_null', 'is_between', 'n_unique', 'cum_sum', 'bottom_k', 'extend_constant', 'rolling_sum',

In [10]:
df=pd.read_csv("/content/sample_data/vehicles.csv")
print(df)

       barrels08  barrelsA08  charge120  charge240  city08  city08U  cityA08  \
0      14.167143         0.0        0.0        0.0      19      0.0        0   
1      27.046364         0.0        0.0        0.0       9      0.0        0   
2      11.018889         0.0        0.0        0.0      23      0.0        0   
3      27.046364         0.0        0.0        0.0      10      0.0        0   
4      15.658421         0.0        0.0        0.0      17      0.0        0   
...          ...         ...        ...        ...     ...      ...      ...   
48576  13.523182         0.0        0.0        0.0      19      0.0        0   
48577  12.935217         0.0        0.0        0.0      20      0.0        0   
48578  14.167143         0.0        0.0        0.0      18      0.0        0   
48579  14.167143         0.0        0.0        0.0      18      0.0        0   
48580  16.528333         0.0        0.0        0.0      16      0.0        0   

       cityA08U  cityCD  cityE  ...  mf

<ipython-input-10-a126a39993e8>:1: DtypeWarning: Columns (74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/sample_data/vehicles.csv")


In [11]:
lazy = pl.scan_csv("/content/sample_data/vehicles.csv", null_values=['NA'])
print(lazy)
lazy.explain(optimized=True)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [/content/sample_data/vehicles.csv]
PROJECT */84 COLUMNS


'Csv SCAN [/content/sample_data/vehicles.csv]\nPROJECT */84 COLUMNS'

In [12]:
print(lazy
 .filter((pl.col('year') >= 1990) & (pl.col('year') < 2000))
 .select(['year','make'])
 .explain()
)

simple π 2/2 ["year", "make"]
  Csv SCAN [/content/sample_data/vehicles.csv]
  PROJECT 2/84 COLUMNS
  SELECTION: [([(col("year")) < (2000)]) & ([(col("year")) >= (1990)])]


In [13]:
print(df.dtypes)

barrels08     float64
barrelsA08    float64
charge120     float64
charge240     float64
city08          int64
               ...   
modifiedOn     object
startStop      object
phevCity        int64
phevHwy         int64
phevComb        int64
Length: 84, dtype: object


In [ ]:
print(df.info(verbose=True))

In [15]:
cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08','createdOn']
df_polars = pl.from_pandas(df)
print(df_polars.select(cols))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i64  ┆ str        ┆ str            ┆ f64   ┆   ┆ f64       ┆ i64    ┆ i64       ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆

In [16]:
print(df_polars.select(pl.col(cols)))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i64  ┆ str        ┆ str            ┆ f64   ┆   ┆ f64       ┆ i64    ┆ i64       ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆

In [17]:
print(df_polars
 .select(pl.col(cols))
 .select(pl.col(pl.Int64))
 .describe())

shape: (9, 4)
┌────────────┬─────────────┬───────────┬───────────┐
│ statistic  ┆ year        ┆ city08    ┆ highway08 │
│ ---        ┆ ---         ┆ ---       ┆ ---       │
│ str        ┆ f64         ┆ f64       ┆ f64       │
╞════════════╪═════════════╪═══════════╪═══════════╡
│ count      ┆ 48581.0     ┆ 48581.0   ┆ 48581.0   │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0       │
│ mean       ┆ 2004.752413 ┆ 20.131615 ┆ 25.970256 │
│ std        ┆ 12.758499   ┆ 13.198585 ┆ 11.439295 │
│ min        ┆ 1984.0      ┆ 6.0       ┆ 9.0       │
│ 25%        ┆ 1993.0      ┆ 15.0      ┆ 20.0      │
│ 50%        ┆ 2006.0      ┆ 18.0      ┆ 24.0      │
│ 75%        ┆ 2016.0      ┆ 21.0      ┆ 28.0      │
│ max        ┆ 2026.0      ┆ 153.0     ┆ 142.0     │
└────────────┴─────────────┴───────────┴───────────┘


In [ ]:
#When the with_columns context runs, Polars will read the data from the year column and cast it to a 16-bit integer and cast it to a 16-bit integer
print(df_polars
 .select(pl.col(cols))
 .with_columns(pl.col('year').cast(pl.Int16),
               pl.col('cylinders').cast(pl.UInt8),
               pl.col('highway08').cast(pl.UInt8),
               pl.col('city08').cast(pl.UInt8)))

In [19]:
#Polars includes a column expression to shrink numeric columns, the shrink_dtype method
print(df_polars.select(pl.col(cols).shrink_dtype()))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i16  ┆ str        ┆ str            ┆ f32   ┆   ┆ f32       ┆ i16    ┆ i16       ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆

In [20]:
print(df_polars
.select(pl.col(cols))
.select(pl.col(pl.Float64))
.sample(n=10, seed=42)
)


shape: (10, 3)
┌───────┬───────────┬───────────┐
│ displ ┆ cylinders ┆ barrels08 │
│ ---   ┆ ---       ┆ ---       │
│ f64   ┆ f64       ┆ f64       │
╞═══════╪═══════════╪═══════════╡
│ 3.0   ┆ 6.0       ┆ 17.500588 │
│ 5.3   ┆ 8.0       ┆ 21.250714 │
│ 2.2   ┆ 4.0       ┆ 15.658421 │
│ 2.2   ┆ 4.0       ┆ 12.39625  │
│ 3.5   ┆ 6.0       ┆ 15.658421 │
│ 5.5   ┆ 8.0       ┆ 18.594375 │
│ 2.0   ┆ 4.0       ┆ 12.935217 │
│ 2.0   ┆ 4.0       ┆ 13.523182 │
│ 4.9   ┆ 6.0       ┆ 21.250714 │
│ 4.0   ┆ 8.0       ┆ 17.500588 │
└───────┴───────────┴───────────┘


In [21]:
print(df_polars
 .select(pl.col(cols))
 .with_columns(pl.col('year').cast(pl.Int16),
               pl.col('cylinders').cast(pl.UInt8),
               pl.col('highway08').cast(pl.UInt8),
               pl.col('city08').cast(pl.UInt8),
               pl.col('displ').cast(pl.Float32),
               pl.col('barrels08').cast(pl.Float32)
               ))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i16  ┆ str        ┆ str            ┆ f32   ┆   ┆ f32       ┆ u8     ┆ u8        ┆ str            │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ Tue Jan 01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00 EST   │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 2013           │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ Tue Jan 01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆

In [22]:
print(df_polars
 .with_columns('trany',
 is_automatic=pl.col('trany')
 .str.to_lowercase()
 .str.contains('automatic'))
 )

shape: (48_581, 85)
┌───────────┬────────────┬───────────┬───────────┬───┬──────────┬─────────┬──────────┬─────────────┐
│ barrels08 ┆ barrelsA08 ┆ charge120 ┆ charge240 ┆ … ┆ phevCity ┆ phevHwy ┆ phevComb ┆ is_automati │
│ ---       ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---      ┆ ---     ┆ ---      ┆ c           │
│ f64       ┆ f64        ┆ f64       ┆ f64       ┆   ┆ i64      ┆ i64     ┆ i64      ┆ ---         │
│           ┆            ┆           ┆           ┆   ┆          ┆         ┆          ┆ bool        │
╞═══════════╪════════════╪═══════════╪═══════════╪═══╪══════════╪═════════╪══════════╪═════════════╡
│ 14.167143 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0        ┆ 0       ┆ 0        ┆ false       │
│ 27.046364 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0        ┆ 0       ┆ 0        ┆ false       │
│ 11.018889 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0        ┆ 0       ┆ 0        ┆ false       │
│ 27.046364 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0        ┆ 0    

In [23]:
print(df_polars.select(pl.col('trany').value_counts(sort=True)))

shape: (41, 1)
┌───────────────────────────┐
│ trany                     │
│ ---                       │
│ struct[2]                 │
╞═══════════════════════════╡
│ {"Automatic 4-spd",11048} │
│ {"Manual 5-spd",8392}     │
│ {"Automatic (S8)",3372}   │
│ {"Automatic (S6)",3361}   │
│ {"Automatic 3-spd",3151}  │
│ …                         │
│ {null,11}                 │
│ {"Automatic (AM-S9)",6}   │
│ {"Automatic (L4)",2}      │
│ {"Automatic (L3)",2}      │
│ {"Automatic (AM-S10)",2}  │
└───────────────────────────┘


In [24]:
print(df_polars
 .filter(pl.col('trany').is_null())
 .select('year', 'make', 'model', 'VClass','trany')
 )


shape: (11, 5)
┌──────┬────────┬────────────────────────┬─────────────────────────────┬───────┐
│ year ┆ make   ┆ model                  ┆ VClass                      ┆ trany │
│ ---  ┆ ---    ┆ ---                    ┆ ---                         ┆ ---   │
│ i64  ┆ str    ┆ str                    ┆ str                         ┆ str   │
╞══════╪════════╪════════════════════════╪═════════════════════════════╪═══════╡
│ 2000 ┆ Nissan ┆ Altra EV               ┆ Midsize Station Wagons      ┆ null  │
│ 2000 ┆ Toyota ┆ RAV4 EV                ┆ Sport Utility Vehicle - 2WD ┆ null  │
│ 2001 ┆ Toyota ┆ RAV4 EV                ┆ Sport Utility Vehicle - 2WD ┆ null  │
│ 2001 ┆ Ford   ┆ Th!nk                  ┆ Two Seaters                 ┆ null  │
│ 2001 ┆ Ford   ┆ Explorer USPS Electric ┆ Sport Utility Vehicle - 2WD ┆ null  │
│ …    ┆ …      ┆ …                      ┆ …                           ┆ …     │
│ 2002 ┆ Toyota ┆ RAV4 EV                ┆ Sport Utility Vehicle - 2WD ┆ null  │
│ 2002 ┆ Ford

In [25]:
#FILL null values in column
print(df_polars.select('trany',
                       is_automatic=pl.col('trany')
                       .str.contains('Automatic').fill_null(True))
)



shape: (48_581, 2)
┌─────────────────┬──────────────┐
│ trany           ┆ is_automatic │
│ ---             ┆ ---          │
│ str             ┆ bool         │
╞═════════════════╪══════════════╡
│ Manual 5-spd    ┆ false        │
│ Manual 5-spd    ┆ false        │
│ Manual 5-spd    ┆ false        │
│ Automatic 3-spd ┆ true         │
│ Manual 5-spd    ┆ false        │
│ …               ┆ …            │
│ Automatic 4-spd ┆ true         │
│ Manual 5-spd    ┆ false        │
│ Automatic 4-spd ┆ true         │
│ Manual 5-spd    ┆ false        │
│ Automatic 4-spd ┆ true         │
└─────────────────┴──────────────┘


In [26]:
#extarct num of gears in trany column and check where it is null
print(df_polars
 .select(num_gears=pl.col('trany')
 .str.extract(r'(\d+)')
 .cast(pl.UInt8))
 .filter(pl.col('num_gears').is_null())
)


shape: (1_155, 1)
┌───────────┐
│ num_gears │
│ ---       │
│ u8        │
╞═══════════╡
│ null      │
│ null      │
│ null      │
│ null      │
│ null      │
│ …         │
│ null      │
│ null      │
│ null      │
│ null      │
│ null      │
└───────────┘


In [27]:
print(df_polars.with_columns(
      is_automatic=pl.col('trany')
                   .str.contains('Automatic')
                   .fill_null(True),
      num_gears=pl.col('trany')
                   .str.extract(r'(\d+)')
                   .cast(pl.UInt8)
                   .fill_null(6)
 )
 )


shape: (48_581, 86)
┌───────────┬────────────┬───────────┬───────────┬───┬─────────┬──────────┬────────────┬───────────┐
│ barrels08 ┆ barrelsA08 ┆ charge120 ┆ charge240 ┆ … ┆ phevHwy ┆ phevComb ┆ is_automat ┆ num_gears │
│ ---       ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---     ┆ ---      ┆ ic         ┆ ---       │
│ f64       ┆ f64        ┆ f64       ┆ f64       ┆   ┆ i64     ┆ i64      ┆ ---        ┆ u8        │
│           ┆            ┆           ┆           ┆   ┆         ┆          ┆ bool       ┆           │
╞═══════════╪════════════╪═══════════╪═══════════╪═══╪═════════╪══════════╪════════════╪═══════════╡
│ 14.167143 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0       ┆ 0        ┆ false      ┆ 5         │
│ 27.046364 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0       ┆ 0        ┆ false      ┆ 5         │
│ 11.018889 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0       ┆ 0        ┆ false      ┆ 5         │
│ 27.046364 ┆ 0.0        ┆ 0.0       ┆ 0.0       ┆ … ┆ 0       ┆ 0     

In [28]:
#now convert string columns into categorical
pl.enable_string_cache()
with pl.StringCache():#StringCache context manager to ensure that the categorical mappings are the same across columns
 print(df_polars
 .select(pl.col(cols))
 .with_columns(pl.col('year').cast(pl.Int16),
               pl.col('cylinders').cast(pl.UInt8),
               pl.col('highway08').cast(pl.UInt8),
               pl.col('city08').cast(pl.UInt8),
               pl.col('displ').cast(pl.Float32),
               pl.col('barrels08').cast(pl.Float32),
               pl.col(['make', 'model', 'VClass', 'drive','fuelType']).cast(pl.Categorical),
               pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y')#convert the createdOn column to a date
               ))

shape: (48_581, 13)
┌──────┬────────────┬────────────────┬───────┬───┬───────────┬────────┬───────────┬────────────────┐
│ year ┆ make       ┆ model          ┆ displ ┆ … ┆ barrels08 ┆ city08 ┆ highway08 ┆ createdOn      │
│ ---  ┆ ---        ┆ ---            ┆ ---   ┆   ┆ ---       ┆ ---    ┆ ---       ┆ ---            │
│ i16  ┆ cat        ┆ cat            ┆ f32   ┆   ┆ f32       ┆ u8     ┆ u8        ┆ datetime[μs]   │
╞══════╪════════════╪════════════════╪═══════╪═══╪═══════════╪════════╪═══════════╪════════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce  ┆ 2.0   ┆ … ┆ 14.167143 ┆ 19     ┆ 25        ┆ 2013-01-01     │
│      ┆            ┆ 2000           ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00       │
│ 1985 ┆ Ferrari    ┆ Testarossa     ┆ 4.9   ┆ … ┆ 27.046364 ┆ 9      ┆ 14        ┆ 2013-01-01     │
│      ┆            ┆                ┆       ┆   ┆           ┆        ┆           ┆ 00:00:00       │
│ 1985 ┆ Dodge      ┆ Charger        ┆ 2.2   ┆ … ┆ 11.018888 ┆ 23     ┆

##wrap all this with function

In [29]:
@pl.StringCache()
def tweak_auto(df):
 cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany',
 'drive', 'VClass', 'fuelType', 'barrels08', 'city08',
 'highway08', 'createdOn']
 return (df.select(pl.col(cols))
           .with_columns(pl.col('year').cast(pl.Int16),
             pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
             pl.col(['displ', 'barrels08']).cast(pl.Float32),
             pl.col(['make', 'model', 'VClass', 'drive','fuelType']).cast(pl.Categorical),
             pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
         is_automatic=pl.col('trany')
                        .str.contains('Automatic')
                        .fill_null('Automatic'),
         num_gears=pl.col('trany')
                     .str.extract(r'(\d+)')
                     .cast(pl.UInt8)
                     .fill_null(6))
 )

In [30]:
print(tweak_auto(df_polars))

shape: (48_581, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

#now check the size after all conversions and with out conversions


In [31]:
tweak_auto(df_polars).estimated_size(unit='mb')

2.9714279174804688

In [32]:
df_polars.select(pl.col(cols)).estimated_size(unit='mb')

7.043133735656738

In [33]:
#summary statistics
print(tweak_auto(df_polars).describe())

shape: (9, 16)
┌────────────┬─────────────┬───────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ statistic  ┆ year        ┆ make  ┆ model ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---        ┆ ---         ┆ ---   ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ str        ┆ f64         ┆ str   ┆ str   ┆   ┆ f64       ┆ str         ┆ ---         ┆ f64       │
│            ┆             ┆       ┆       ┆   ┆           ┆             ┆ str         ┆           │
╞════════════╪═════════════╪═══════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ count      ┆ 48581.0     ┆ 48581 ┆ 48581 ┆ … ┆ 48581.0   ┆ 48581       ┆ 48581       ┆ 48581.0   │
│ null_count ┆ 0.0         ┆ 0     ┆ 0     ┆ … ┆ 0.0       ┆ 0           ┆ 0           ┆ 0.0       │
│ mean       ┆ 2004.752413 ┆ null  ┆ null  ┆ … ┆ 25.970256 ┆ 2014-12-13  ┆ null        ┆ 5.349231  │
│            ┆             ┆       ┆       ┆   ┆           ┆ 09:38:57.13 ┆  

In [34]:
#.corr method returns a dataframe with the correlation coefficients.However, if a column has missing values, it will return NaN for that column
import polars.selectors as cs
print(tweak_auto(df_polars)
 .select(cs.numeric())
 .corr()
 .pipe(lambda x: x.insert_column(0, pl.Series('names', x.columns)))
)

shape: (7, 8)
┌───────────┬───────────┬───────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ names     ┆ year      ┆ displ ┆ cylinders ┆ barrels08 ┆ city08    ┆ highway08 ┆ num_gears │
│ ---       ┆ ---       ┆ ---   ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ f64       ┆ f64   ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ year      ┆ 1.0       ┆ NaN   ┆ NaN       ┆ -0.347701 ┆ 0.275523  ┆ 0.318163  ┆ 0.641837  │
│ displ     ┆ NaN       ┆ NaN   ┆ NaN       ┆ NaN       ┆ NaN       ┆ NaN       ┆ NaN       │
│ cylinders ┆ NaN       ┆ NaN   ┆ NaN       ┆ NaN       ┆ NaN       ┆ NaN       ┆ NaN       │
│ barrels08 ┆ -0.347701 ┆ NaN   ┆ NaN       ┆ 1.0       ┆ -0.733175 ┆ -0.813099 ┆ -0.017943 │
│ city08    ┆ 0.275523  ┆ NaN   ┆ NaN       ┆ -0.733175 ┆ 1.0       ┆ 0.967449  ┆ -0.252527 │
│ highway08 ┆ 0.318163  ┆ NaN   ┆ NaN       ┆ 

In [35]:
raw = pl.read_csv("/content/sample_data/vehicles.csv", null_values=['NA'])
autos = tweak_auto(raw)
print(autos)


shape: (48_581, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

In [36]:
#Adding new columns
print(autos.select(pl.col(['highway08', 'city08']),
                   mpg_ratio=(pl.col('highway08') / pl.col('city08')))
)

shape: (48_581, 3)
┌───────────┬────────┬───────────┐
│ highway08 ┆ city08 ┆ mpg_ratio │
│ ---       ┆ ---    ┆ ---       │
│ u8        ┆ u8     ┆ f64       │
╞═══════════╪════════╪═══════════╡
│ 25        ┆ 19     ┆ 1.315789  │
│ 14        ┆ 9      ┆ 1.555556  │
│ 33        ┆ 23     ┆ 1.434783  │
│ 12        ┆ 10     ┆ 1.2       │
│ 23        ┆ 17     ┆ 1.352941  │
│ …         ┆ …      ┆ …         │
│ 26        ┆ 19     ┆ 1.368421  │
│ 28        ┆ 20     ┆ 1.4       │
│ 24        ┆ 18     ┆ 1.333333  │
│ 24        ┆ 18     ┆ 1.333333  │
│ 21        ┆ 16     ┆ 1.3125    │
└───────────┴────────┴───────────┘


In [37]:
print(autos.with_columns(mpg_ratio=pl.col('highway08') / pl.col('city08')))

shape: (48_581, 16)
┌──────┬────────────┬──────────────┬───────┬───┬─────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ mpg_ratio │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ f64       │
│      ┆            ┆              ┆       ┆   ┆ ]           ┆ str         ┆           ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 1.315789  │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 1.555556  │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00    ┆         

In [38]:
#drop column from dataframe
print(autos.drop('createdOn').columns)

['year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'is_automatic', 'num_gears']


In [39]:
 print(autos
 .select(pl.all()
 .exclude(['createdOn', 'barrels08'])
 .name.suffix('auto'))
 )


shape: (48_581, 13)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ yearauto ┆ makeauto  ┆ modelauto ┆ displauto ┆ … ┆ city08aut ┆ highway08 ┆ is_automa ┆ num_gears │
│ ---      ┆ ---       ┆ ---       ┆ ---       ┆   ┆ o         ┆ auto      ┆ ticauto   ┆ auto      │
│ i16      ┆ cat       ┆ cat       ┆ f32       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│          ┆           ┆           ┆           ┆   ┆ u8        ┆ u8        ┆ str       ┆ u8        │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 1985     ┆ Alfa      ┆ Spider    ┆ 2.0       ┆ … ┆ 19        ┆ 25        ┆ false     ┆ 5         │
│          ┆ Romeo     ┆ Veloce    ┆           ┆   ┆           ┆           ┆           ┆           │
│          ┆           ┆ 2000      ┆           ┆   ┆           ┆           ┆           ┆           │
│ 1985     ┆ Ferrari   ┆ Testaross ┆ 4.9       ┆ … ┆ 9         ┆ 14    

In [40]:
#renaming columns
final_cols = ['year', 'make', 'model', 'city_mpg', 'highway_mpg']
print(autos.with_columns(pl.col('city08').alias('city_mpg'), highway_mpg=pl.col('highway08')).select(final_cols))

shape: (48_581, 5)
┌──────┬────────────┬─────────────────────┬──────────┬─────────────┐
│ year ┆ make       ┆ model               ┆ city_mpg ┆ highway_mpg │
│ ---  ┆ ---        ┆ ---                 ┆ ---      ┆ ---         │
│ i16  ┆ cat        ┆ cat                 ┆ u8       ┆ u8          │
╞══════╪════════════╪═════════════════════╪══════════╪═════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce 2000  ┆ 19       ┆ 25          │
│ 1985 ┆ Ferrari    ┆ Testarossa          ┆ 9        ┆ 14          │
│ 1985 ┆ Dodge      ┆ Charger             ┆ 23       ┆ 33          │
│ 1985 ┆ Dodge      ┆ B150/B250 Wagon 2WD ┆ 10       ┆ 12          │
│ 1993 ┆ Subaru     ┆ Legacy AWD Turbo    ┆ 17       ┆ 23          │
│ …    ┆ …          ┆ …                   ┆ …        ┆ …           │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 19       ┆ 26          │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 20       ┆ 28          │
│ 1993 ┆ Subaru     ┆ Legacy AWD          ┆ 18       ┆ 24          │
│ 1993 ┆ Subaru

In [41]:
#we can use rename also--accept dict mapping
print(autos
 .rename({'city08':'city_mpg',
 'highway08':'highway_mpg'})
 .select(final_cols))

shape: (48_581, 5)
┌──────┬────────────┬─────────────────────┬──────────┬─────────────┐
│ year ┆ make       ┆ model               ┆ city_mpg ┆ highway_mpg │
│ ---  ┆ ---        ┆ ---                 ┆ ---      ┆ ---         │
│ i16  ┆ cat        ┆ cat                 ┆ u8       ┆ u8          │
╞══════╪════════════╪═════════════════════╪══════════╪═════════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider Veloce 2000  ┆ 19       ┆ 25          │
│ 1985 ┆ Ferrari    ┆ Testarossa          ┆ 9        ┆ 14          │
│ 1985 ┆ Dodge      ┆ Charger             ┆ 23       ┆ 33          │
│ 1985 ┆ Dodge      ┆ B150/B250 Wagon 2WD ┆ 10       ┆ 12          │
│ 1993 ┆ Subaru     ┆ Legacy AWD Turbo    ┆ 17       ┆ 23          │
│ …    ┆ …          ┆ …                   ┆ …        ┆ …           │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 19       ┆ 26          │
│ 1993 ┆ Subaru     ┆ Legacy              ┆ 20       ┆ 28          │
│ 1993 ┆ Subaru     ┆ Legacy AWD          ┆ 18       ┆ 24          │
│ 1993 ┆ Subaru

##Joins

In [42]:
trucks = pl.DataFrame(
 {'make': ['Ford', 'Tesla', 'Chevy', 'Custom', 'Ford'],
 'model': ['F150', 'Cybertruck', 'Silverado', 'HotRod', 'F250'],
 'year': [2018, 2024, 2019, 1967, 2017],
 'city_mpg': [19, None, 17, 12, 18],
 })
print(trucks)


shape: (5, 4)
┌────────┬────────────┬──────┬──────────┐
│ make   ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str        ┆ i64  ┆ i64      │
╞════════╪════════════╪══════╪══════════╡
│ Ford   ┆ F150       ┆ 2018 ┆ 19       │
│ Tesla  ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy  ┆ Silverado  ┆ 2019 ┆ 17       │
│ Custom ┆ HotRod     ┆ 1967 ┆ 12       │
│ Ford   ┆ F250       ┆ 2017 ┆ 18       │
└────────┴────────────┴──────┴──────────┘


In [43]:
manufacturer = pl.DataFrame(
 {'name': ['Ford', 'Tesla', 'Chevy', 'Toyota'],
 'country': ['USA', 'USA', 'USA', 'Japan'],
 'founded': [1903, 2003, 1911, 1937],
 'employees': [199_000, 48_000, 225_000, 370_000],
 'vehicles': [80, 3, 45, 30],
 })
print(manufacturer)

shape: (4, 5)
┌────────┬─────────┬─────────┬───────────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ vehicles │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       │
│ Tesla  ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        │
│ Chevy  ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       │
│ Toyota ┆ Japan   ┆ 1937    ┆ 370000    ┆ 30       │
└────────┴─────────┴─────────┴───────────┴──────────┘


In [44]:
print(manufacturer.join(trucks, how='left', left_on='name', right_on='make'))

shape: (5, 8)
┌────────┬─────────┬─────────┬───────────┬──────────┬────────────┬──────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ vehicles ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆ ---      ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆ i64      ┆ str        ┆ i64  ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪══════════╪════════════╪══════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F150       ┆ 2018 ┆ 19       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F250       ┆ 2017 ┆ 18       │
│ Tesla  ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy  ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       ┆ Silverado  ┆ 2019 ┆ 17       │
│ Toyota ┆ Japan   ┆ 1937    ┆ 370000    ┆ 30       ┆ null       ┆ null ┆ null     │
└────────┴─────────┴─────────┴───────────┴──────────┴────────────┴──────┴──────────┘


Polars doesn’t support how='right' in the .join method. Instead, we can
use how='left' and then swap the dataframes. (We will also need to swap the
left_on and right_on parameters.)


In [45]:
print(trucks.join(manufacturer, how='left', left_on='make', right_on='name'))

shape: (5, 8)
┌────────┬────────────┬──────┬──────────┬─────────┬─────────┬───────────┬──────────┐
│ make   ┆ model      ┆ year ┆ city_mpg ┆ country ┆ founded ┆ employees ┆ vehicles │
│ ---    ┆ ---        ┆ ---  ┆ ---      ┆ ---     ┆ ---     ┆ ---       ┆ ---      │
│ str    ┆ str        ┆ i64  ┆ i64      ┆ str     ┆ i64     ┆ i64       ┆ i64      │
╞════════╪════════════╪══════╪══════════╪═════════╪═════════╪═══════════╪══════════╡
│ Ford   ┆ F150       ┆ 2018 ┆ 19       ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       │
│ Tesla  ┆ Cybertruck ┆ 2024 ┆ null     ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        │
│ Chevy  ┆ Silverado  ┆ 2019 ┆ 17       ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       │
│ Custom ┆ HotRod     ┆ 1967 ┆ 12       ┆ null    ┆ null    ┆ null      ┆ null     │
│ Ford   ┆ F250       ┆ 2017 ┆ 18       ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       │
└────────┴────────────┴──────┴──────────┴─────────┴─────────┴───────────┴──────────┘


In [46]:
##inner join--it is default join
print(manufacturer.join(trucks, left_on='name', right_on='make'))

shape: (4, 8)
┌───────┬─────────┬─────────┬───────────┬──────────┬────────────┬──────┬──────────┐
│ name  ┆ country ┆ founded ┆ employees ┆ vehicles ┆ model      ┆ year ┆ city_mpg │
│ ---   ┆ ---     ┆ ---     ┆ ---       ┆ ---      ┆ ---        ┆ ---  ┆ ---      │
│ str   ┆ str     ┆ i64     ┆ i64       ┆ i64      ┆ str        ┆ i64  ┆ i64      │
╞═══════╪═════════╪═════════╪═══════════╪══════════╪════════════╪══════╪══════════╡
│ Ford  ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F150       ┆ 2018 ┆ 19       │
│ Tesla ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       ┆ Silverado  ┆ 2019 ┆ 17       │
│ Ford  ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F250       ┆ 2017 ┆ 18       │
└───────┴─────────┴─────────┴───────────┴──────────┴────────────┴──────┴──────────┘


In [47]:
##outer join
print(manufacturer.join(trucks, how='outer', left_on='name', right_on='make'))

shape: (6, 9)
┌────────┬─────────┬─────────┬───────────┬───┬────────┬────────────┬──────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ … ┆ make   ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---    ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆   ┆ str    ┆ str        ┆ i64  ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪═══╪════════╪════════════╪══════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Ford   ┆ F150       ┆ 2018 ┆ 19       │
│ Tesla  ┆ USA     ┆ 2003    ┆ 48000     ┆ … ┆ Tesla  ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy  ┆ USA     ┆ 1911    ┆ 225000    ┆ … ┆ Chevy  ┆ Silverado  ┆ 2019 ┆ 17       │
│ null   ┆ null    ┆ null    ┆ null      ┆ … ┆ Custom ┆ HotRod     ┆ 1967 ┆ 12       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Ford   ┆ F250       ┆ 2017 ┆ 18       │
│ Toyota ┆ Japan   ┆ 1937    ┆ 370000    ┆ … ┆ null   ┆ null       ┆ null ┆ null     │
└────────┴─────────┴─────────

<ipython-input-47-dcaebdaff23b>:2: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  print(manufacturer.join(trucks, how='outer', left_on='name', right_on='make'))


In [48]:
#semi join
print(manufacturer.join(trucks, how='semi', left_on='name', right_on='make'))

shape: (3, 5)
┌───────┬─────────┬─────────┬───────────┬──────────┐
│ name  ┆ country ┆ founded ┆ employees ┆ vehicles │
│ ---   ┆ ---     ┆ ---     ┆ ---       ┆ ---      │
│ str   ┆ str     ┆ i64     ┆ i64       ┆ i64      │
╞═══════╪═════════╪═════════╪═══════════╪══════════╡
│ Ford  ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       │
│ Tesla ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        │
│ Chevy ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       │
└───────┴─────────┴─────────┴───────────┴──────────┘


In [49]:
#cross join
print(manufacturer.join(trucks, how='cross'))

shape: (20, 9)
┌────────┬─────────┬─────────┬───────────┬───┬────────┬────────────┬──────┬──────────┐
│ name   ┆ country ┆ founded ┆ employees ┆ … ┆ make   ┆ model      ┆ year ┆ city_mpg │
│ ---    ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---    ┆ ---        ┆ ---  ┆ ---      │
│ str    ┆ str     ┆ i64     ┆ i64       ┆   ┆ str    ┆ str        ┆ i64  ┆ i64      │
╞════════╪═════════╪═════════╪═══════════╪═══╪════════╪════════════╪══════╪══════════╡
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Ford   ┆ F150       ┆ 2018 ┆ 19       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Tesla  ┆ Cybertruck ┆ 2024 ┆ null     │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Chevy  ┆ Silverado  ┆ 2019 ┆ 17       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Custom ┆ HotRod     ┆ 1967 ┆ 12       │
│ Ford   ┆ USA     ┆ 1903    ┆ 199000    ┆ … ┆ Ford   ┆ F250       ┆ 2017 ┆ 18       │
│ …      ┆ …       ┆ …       ┆ …         ┆ … ┆ …      ┆ …          ┆ …    ┆ …        │
│ Toyota ┆ Japan   ┆ 1937   

In [50]:
#anti-join- keeps all of the rows from the left dataframe that don’t have amatch in the right dataframe
print(trucks.join(manufacturer, how='anti', left_on='make', right_on='name'))

shape: (1, 4)
┌────────┬────────┬──────┬──────────┐
│ make   ┆ model  ┆ year ┆ city_mpg │
│ ---    ┆ ---    ┆ ---  ┆ ---      │
│ str    ┆ str    ┆ i64  ┆ i64      │
╞════════╪════════╪══════╪══════════╡
│ Custom ┆ HotRod ┆ 1967 ┆ 12       │
└────────┴────────┴──────┴──────────┘


In [51]:
#validating join types
print(manufacturer.join(trucks, left_on='name', right_on='make', validate='1:m'))

shape: (4, 8)
┌───────┬─────────┬─────────┬───────────┬──────────┬────────────┬──────┬──────────┐
│ name  ┆ country ┆ founded ┆ employees ┆ vehicles ┆ model      ┆ year ┆ city_mpg │
│ ---   ┆ ---     ┆ ---     ┆ ---       ┆ ---      ┆ ---        ┆ ---  ┆ ---      │
│ str   ┆ str     ┆ i64     ┆ i64       ┆ i64      ┆ str        ┆ i64  ┆ i64      │
╞═══════╪═════════╪═════════╪═══════════╪══════════╪════════════╪══════╪══════════╡
│ Ford  ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F150       ┆ 2018 ┆ 19       │
│ Tesla ┆ USA     ┆ 2003    ┆ 48000     ┆ 3        ┆ Cybertruck ┆ 2024 ┆ null     │
│ Chevy ┆ USA     ┆ 1911    ┆ 225000    ┆ 45       ┆ Silverado  ┆ 2019 ┆ 17       │
│ Ford  ┆ USA     ┆ 1903    ┆ 199000    ┆ 80       ┆ F250       ┆ 2017 ┆ 18       │
└───────┴─────────┴─────────┴───────────┴──────────┴────────────┴──────┴──────────┘


In [52]:
print(trucks.filter(pl.col('make').is_duplicated()))

shape: (2, 4)
┌──────┬───────┬──────┬──────────┐
│ make ┆ model ┆ year ┆ city_mpg │
│ ---  ┆ ---   ┆ ---  ┆ ---      │
│ str  ┆ str   ┆ i64  ┆ i64      │
╞══════╪═══════╪══════╪══════════╡
│ Ford ┆ F150  ┆ 2018 ┆ 19       │
│ Ford ┆ F250  ┆ 2017 ┆ 18       │
└──────┴───────┴──────┴──────────┘


In [53]:
 #filter out the top 5 car manufacturers
 top_n = (autos
 .group_by('make')
 .agg(pl.col('city08').count())
 .sort('city08')
 .tail(5)
 )
 print(top_n)

shape: (5, 2)
┌───────────┬────────┐
│ make      ┆ city08 │
│ ---       ┆ ---    │
│ cat       ┆ u32    │
╞═══════════╪════════╡
│ BMW       ┆ 2545   │
│ Dodge     ┆ 2703   │
│ GMC       ┆ 2836   │
│ Ford      ┆ 3867   │
│ Chevrolet ┆ 4490   │
└───────────┴────────┘


In [56]:
#pivot
print(autos
 .filter(pl.col('make').is_in(top_n['make']))
 .sort('year')
 .pivot(on='make', index='year', values='city08',
 aggregate_function='median',maintain_order=True)#indicate that results should be in the order of the index column by using maintain_order=True
)

shape: (42, 6)
┌──────┬───────────┬──────┬──────┬───────┬──────┐
│ year ┆ Chevrolet ┆ Ford ┆ BMW  ┆ Dodge ┆ GMC  │
│ ---  ┆ ---       ┆ ---  ┆ ---  ┆ ---   ┆ ---  │
│ i16  ┆ f64       ┆ f64  ┆ f64  ┆ f64   ┆ f64  │
╞══════╪═══════════╪══════╪══════╪═══════╪══════╡
│ 1984 ┆ 16.0      ┆ 14.5 ┆ 18.0 ┆ 18.0  ┆ 15.0 │
│ 1985 ┆ 15.0      ┆ 15.0 ┆ 18.0 ┆ 17.0  ┆ 15.0 │
│ 1986 ┆ 15.0      ┆ 16.0 ┆ 15.0 ┆ 14.5  ┆ 15.0 │
│ 1987 ┆ 15.0      ┆ 15.0 ┆ 15.0 ┆ 15.0  ┆ 15.0 │
│ 1988 ┆ 15.0      ┆ 15.0 ┆ 15.0 ┆ 14.0  ┆ 15.0 │
│ …    ┆ …         ┆ …    ┆ …    ┆ …     ┆ …    │
│ 2021 ┆ 16.0      ┆ 20.0 ┆ 21.0 ┆ 15.0  ┆ 15.0 │
│ 2022 ┆ 16.0      ┆ 20.0 ┆ 21.0 ┆ 15.0  ┆ 15.0 │
│ 2023 ┆ 17.0      ┆ 19.0 ┆ 22.0 ┆ 15.0  ┆ 16.0 │
│ 2024 ┆ 17.0      ┆ 19.0 ┆ 23.0 ┆ 21.0  ┆ 17.0 │
│ 2025 ┆ 17.0      ┆ 19.5 ┆ 25.0 ┆ 18.0  ┆ 17.0 │
└──────┴───────────┴──────┴──────┴───────┴──────┘


In [62]:
pivoted=autos.filter(pl.col('make').is_in(top_n['make'])).sort('year').pivot(on='make', index='year', values='city08',
                          aggregate_function='median',maintain_order=True)

In [70]:
#unpivot
print(pivoted.unpivot(index='year', on=['Chevrolet', 'Ford', 'GMC', 'Dodge'],
       variable_name='make',value_name='median_city_mpg'))#variable_name and value_name parameters can be used to give names for melted columns


shape: (168, 3)
┌──────┬───────────┬─────────────────┐
│ year ┆ make      ┆ median_city_mpg │
│ ---  ┆ ---       ┆ ---             │
│ i16  ┆ str       ┆ f64             │
╞══════╪═══════════╪═════════════════╡
│ 1984 ┆ Chevrolet ┆ 16.0            │
│ 1985 ┆ Chevrolet ┆ 15.0            │
│ 1986 ┆ Chevrolet ┆ 15.0            │
│ 1987 ┆ Chevrolet ┆ 15.0            │
│ 1988 ┆ Chevrolet ┆ 15.0            │
│ …    ┆ …         ┆ …               │
│ 2021 ┆ Dodge     ┆ 15.0            │
│ 2022 ┆ Dodge     ┆ 15.0            │
│ 2023 ┆ Dodge     ┆ 15.0            │
│ 2024 ┆ Dodge     ┆ 21.0            │
│ 2025 ┆ Dodge     ┆ 18.0            │
└──────┴───────────┴─────────────────┘


In [71]:
#In Polars, a struct is a single column that contains a sequence of values (the collection of columns that are combined)
#below combines two columns
print(autos
 .select(pl.struct(pl.col('year'), pl.col('model')))
 )


shape: (48_581, 1)
┌──────────────────────────────┐
│ year                         │
│ ---                          │
│ struct[2]                    │
╞══════════════════════════════╡
│ {1985,"Spider Veloce 2000"}  │
│ {1985,"Testarossa"}          │
│ {1985,"Charger"}             │
│ {1985,"B150/B250 Wagon 2WD"} │
│ {1993,"Legacy AWD Turbo"}    │
│ …                            │
│ {1993,"Legacy"}              │
│ {1993,"Legacy"}              │
│ {1993,"Legacy AWD"}          │
│ {1993,"Legacy AWD"}          │
│ {1993,"Legacy AWD Turbo"}    │
└──────────────────────────────┘


In [72]:
#below combines two columns and check there is any duplicates
print(autos
 .select(pl.struct(pl.col('year'), pl.col('model')).is_duplicated())
 )

shape: (48_581, 1)
┌───────┐
│ year  │
│ ---   │
│ bool  │
╞═══════╡
│ false │
│ false │
│ true  │
│ true  │
│ true  │
│ …     │
│ true  │
│ true  │
│ true  │
│ true  │
│ true  │
└───────┘


In [74]:
#we can see those combination dupliactes using filter
print(autos
 .filter(pl.struct(pl.col('year'), pl.col('model')).is_duplicated())
 )

shape: (35_143, 15)
┌──────┬────────┬────────────┬───────┬───┬───────────┬──────────────┬──────────────┬───────────┐
│ year ┆ make   ┆ model      ┆ displ ┆ … ┆ highway08 ┆ createdOn    ┆ is_automatic ┆ num_gears │
│ ---  ┆ ---    ┆ ---        ┆ ---   ┆   ┆ ---       ┆ ---          ┆ ---          ┆ ---       │
│ i16  ┆ cat    ┆ cat        ┆ f32   ┆   ┆ u8        ┆ datetime[μs] ┆ str          ┆ u8        │
╞══════╪════════╪════════════╪═══════╪═══╪═══════════╪══════════════╪══════════════╪═══════════╡
│ 1985 ┆ Dodge  ┆ Charger    ┆ 2.2   ┆ … ┆ 33        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆        ┆            ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1985 ┆ Dodge  ┆ B150/B250  ┆ 5.2   ┆ … ┆ 12        ┆ 2013-01-01   ┆ true         ┆ 3         │
│      ┆        ┆ Wagon 2WD  ┆       ┆   ┆           ┆ 00:00:00     ┆              ┆           │
│ 1993 ┆ Subaru ┆ Legacy AWD ┆ 2.2   ┆ … ┆ 23        ┆ 2013-01-01   ┆ false        ┆ 5         │
│      ┆  

#handling nulls and NaN values and filing null values

In [75]:
missing_df = pl.DataFrame({'val': [-1.1, 0, 2.3, None, 5.7, 7]})
print(missing_df)

shape: (6, 1)
┌──────┐
│ val  │
│ ---  │
│ f64  │
╞══════╡
│ -1.1 │
│ 0.0  │
│ 2.3  │
│ null │
│ 5.7  │
│ 7.0  │
└──────┘


In [76]:
print(missing_df
 .with_columns(val2=pl.col('val')/pl.col('val'))
 .with_columns(is_null2=pl.col('val2').is_null(),
 is_nan2=pl.col('val2').is_nan(),
 is_finite2=pl.col('val2').is_finite(),
 interpolate=pl.col('val2').interpolate()
 )
 )


shape: (6, 6)
┌──────┬──────┬──────────┬─────────┬────────────┬─────────────┐
│ val  ┆ val2 ┆ is_null2 ┆ is_nan2 ┆ is_finite2 ┆ interpolate │
│ ---  ┆ ---  ┆ ---      ┆ ---     ┆ ---        ┆ ---         │
│ f64  ┆ f64  ┆ bool     ┆ bool    ┆ bool       ┆ f64         │
╞══════╪══════╪══════════╪═════════╪════════════╪═════════════╡
│ -1.1 ┆ 1.0  ┆ false    ┆ false   ┆ true       ┆ 1.0         │
│ 0.0  ┆ NaN  ┆ false    ┆ true    ┆ false      ┆ NaN         │
│ 2.3  ┆ 1.0  ┆ false    ┆ false   ┆ true       ┆ 1.0         │
│ null ┆ null ┆ true     ┆ null    ┆ null       ┆ 1.0         │
│ 5.7  ┆ 1.0  ┆ false    ┆ false   ┆ true       ┆ 1.0         │
│ 7.0  ┆ 1.0  ┆ false    ┆ false   ┆ true       ┆ 1.0         │
└──────┴──────┴──────────┴─────────┴────────────┴─────────────┘


In [77]:
#fill_null method replace null values
print(missing_df
.with_columns(
 forward=pl.col('val').fill_null(strategy='forward'),
 backward=pl.col('val').fill_null(strategy='backward'),
 min=pl.col('val').fill_null(strategy='min'),
 max=pl.col('val').fill_null(strategy='max')
 )
 )


shape: (6, 5)
┌──────┬─────────┬──────────┬──────┬──────┐
│ val  ┆ forward ┆ backward ┆ min  ┆ max  │
│ ---  ┆ ---     ┆ ---      ┆ ---  ┆ ---  │
│ f64  ┆ f64     ┆ f64      ┆ f64  ┆ f64  │
╞══════╪═════════╪══════════╪══════╪══════╡
│ -1.1 ┆ -1.1    ┆ -1.1     ┆ -1.1 ┆ -1.1 │
│ 0.0  ┆ 0.0     ┆ 0.0      ┆ 0.0  ┆ 0.0  │
│ 2.3  ┆ 2.3     ┆ 2.3      ┆ 2.3  ┆ 2.3  │
│ null ┆ 2.3     ┆ 5.7      ┆ -1.1 ┆ 7.0  │
│ 5.7  ┆ 5.7     ┆ 5.7      ┆ 5.7  ┆ 5.7  │
│ 7.0  ┆ 7.0     ┆ 7.0      ┆ 7.0  ┆ 7.0  │
└──────┴─────────┴──────────┴──────┴──────┘


In [81]:
#selectors module to select the numeric columns
import polars.selectors as cs
print(autos.select(cs.numeric().is_null().sum()))
#You can also perform set operations on the selectors

shape: (1, 7)
┌──────┬───────┬───────────┬───────────┬────────┬───────────┬───────────┐
│ year ┆ displ ┆ cylinders ┆ barrels08 ┆ city08 ┆ highway08 ┆ num_gears │
│ ---  ┆ ---   ┆ ---       ┆ ---       ┆ ---    ┆ ---       ┆ ---       │
│ u32  ┆ u32   ┆ u32       ┆ u32       ┆ u32    ┆ u32       ┆ u32       │
╞══════╪═══════╪═══════════╪═══════════╪════════╪═══════════╪═══════════╡
│ 0    ┆ 1088  ┆ 1090      ┆ 0         ┆ 0      ┆ 0         ┆ 0         │
└──────┴───────┴───────────┴───────────┴────────┴───────────┴───────────┘


In [83]:
# if you want to select all columns that are not dates orbooleans, use this code
no_dates_or_bools = (cs.all() - cs.date() - cs.boolean())
print(no_dates_or_bools)
print(cs.expand_selector(autos, no_dates_or_bools))


selector
('year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn', 'is_automatic', 'num_gears')


In [85]:
mean_displ = (pl.col('displ')
 .mean().over(['year', 'make'])
 .fill_null(0)
 .alias('mean_displ')
 )
print(autos
 .with_columns(mean_displ)
 .filter(pl.col('displ').is_null())
 .select(['year', 'make', 'displ', 'mean_displ'])
 )


shape: (1_088, 4)
┌──────┬────────┬───────┬────────────┐
│ year ┆ make   ┆ displ ┆ mean_displ │
│ ---  ┆ ---    ┆ ---   ┆ ---        │
│ i16  ┆ cat    ┆ f32   ┆ f32        │
╞══════╪════════╪═══════╪════════════╡
│ 2000 ┆ Nissan ┆ null  ┆ 2.836      │
│ 2000 ┆ Toyota ┆ null  ┆ 2.644444   │
│ 2001 ┆ Toyota ┆ null  ┆ 2.736538   │
│ 2001 ┆ Ford   ┆ null  ┆ 3.837362   │
│ 2001 ┆ Ford   ┆ null  ┆ 3.837362   │
│ …    ┆ …      ┆ …     ┆ …          │
│ 2025 ┆ Audi   ┆ null  ┆ 2.726471   │
│ 2025 ┆ Dodge  ┆ null  ┆ 4.014286   │
│ 2025 ┆ Dodge  ┆ null  ┆ 4.014286   │
│ 2025 ┆ Tesla  ┆ null  ┆ 0.0        │
│ 2025 ┆ Toyota ┆ null  ┆ 2.439743   │
└──────┴────────┴───────┴────────────┘


In [88]:
print(autos
 .with_columns(pl.col('displ').fill_null(mean_displ))
 .select(['year', 'make', 'model', 'displ'])
 .filter(pl.col('make')=='Tesla')
 )


shape: (160, 4)
┌──────┬───────┬─────────────────────────────────┬───────┐
│ year ┆ make  ┆ model                           ┆ displ │
│ ---  ┆ ---   ┆ ---                             ┆ ---   │
│ i16  ┆ cat   ┆ cat                             ┆ f32   │
╞══════╪═══════╪═════════════════════════════════╪═══════╡
│ 2012 ┆ Tesla ┆ Model S                         ┆ 0.0   │
│ 2013 ┆ Tesla ┆ Model S (60 kW-hr battery pack… ┆ 0.0   │
│ 2013 ┆ Tesla ┆ Model S (85 kW-hr battery pack… ┆ 0.0   │
│ 2013 ┆ Tesla ┆ Model S (40 kW-hr battery pack… ┆ 0.0   │
│ 2014 ┆ Tesla ┆ Model S (85 kW-hr battery pack… ┆ 0.0   │
│ …    ┆ …     ┆ …                               ┆ …     │
│ 2025 ┆ Tesla ┆ Model Y Long Range RWD          ┆ 0.0   │
│ 2025 ┆ Tesla ┆ Model Y Performance AWD         ┆ 0.0   │
│ 2024 ┆ Tesla ┆ Model 3 Long Range RWD          ┆ 0.0   │
│ 2024 ┆ Tesla ┆ Model 3 Performance AWD         ┆ 0.0   │
│ 2025 ┆ Tesla ┆ Model 3 Performance AWD         ┆ 0.0   │
└──────┴───────┴────────────────────────

In [91]:
#make a  function that standardizes a column.
def standardize_pl(col):
   return (col - col.mean()) / col.std()
print(autos
 .select(city08_standardized=standardize_pl(pl.col('city08'))))

shape: (48_581, 1)
┌─────────────────────┐
│ city08_standardized │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ -0.085738           │
│ -0.843395           │
│ 0.217325            │
│ -0.767629           │
│ -0.237269           │
│ …                   │
│ -0.085738           │
│ -0.009972           │
│ -0.161503           │
│ -0.161503           │
│ -0.313035           │
└─────────────────────┘
